<a href="https://colab.research.google.com/github/lukezhq10/fcc-predict-health-costs-with-regression/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.model_selection import train_test_split

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
df = pd.read_csv('insurance.csv')
df.tail()

# Check missing values

In [ ]:
# check null values
df.isnull().sum()

# Convert categorical data to numbers

## Region

In [ ]:
df.region.value_counts()

In [ ]:
df = pd.get_dummies(df, columns=['region'])
df.head()

## Sex

In [ ]:
df.sex.value_counts()

In [ ]:
sex_mapping = {'female': 0, 'male': 1}
df['sex'] = df['sex'].map(sex_mapping)
df.head()

In [ ]:
df.sex.value_counts()

## Smoker

In [ ]:
df.smoker.value_counts()

In [ ]:
smoker_mapping = {'no': 0, 'yes': 1}
df['smoker'] = df['smoker'].map(smoker_mapping)
df.head()

# Create train_dataset and test_dataset

In [ ]:
train_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=42)
train_dataset.value_counts()

In [ ]:
test_dataset.value_counts()

# Create train_labels and test_labels

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
train_labels.head()

In [ ]:
test_labels.head()

# Create and train model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(len(train_dataset.keys()))),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=0.05),
    loss='mse',
    metrics=['mae', 'mse']
)
model.summary()

In [ ]:
# taken from tfdocs.modeling.EpochDots
# prints progress dots during training process at every 100th epoch
class EpochDots(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0:
      print()
      print('Epoch: {:d}, '.format(epoch), end='')
      for name, value in sorted(logs.items()):
        print('{}:{:0.4f}'.format(name, value), end=', ')
      print()

    print('.', end='')

In [ ]:
# Train model
history = model.fit(train_dataset, train_labels, epochs=500, batch_size=32, validation_split=0.2, verbose=0, callbacks=[EpochDots()])

In [ ]:
# Evaluate model on train dataset
train_loss, train_mae, train_mse = model.evaluate(train_dataset, train_labels)
print(train_loss, train_mae, train_mse)

# Test

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
